# Your Final Project Title

- Keeley: put their contribution here
- Goar: put their contribution here
- Isaac: put their contribution here
- Jim: put their contribution here
- Vik: put their contribution here 


# Introduction

### This project is about analyzing Twitter tweets' sentiments about a selection of video games. The video games to be analyzed are: 
##### Borderlands, Call of Duty Black Ops Cold War, Overwatch, NBA2k, Dota 2, World of War Craft, CS-GO, Assasin's Creed, Apex Legends, League of Legends, Fortnite, Battlefield, Heartstone, PUB G, FIFA, RDR, Rainbow Six, GTA, Madden NFL, Cyberpunk 2077, Ghost Recon, and Hearthstone. 

# Research Question
##### Which video game will have the most positive sentiment analysis on Twitter?
##### Which video game will have the most negative sentiment analysis?


# Dataset Description

In [ ]:
# Install necessary libraries

In [16]:
!pip install nltk
!pip install -U textblob

In [17]:
# import necessary functions
import pandas as pd
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from textblob import TextBlob
import re

#### df.describe

In [18]:
vg = pd.read_csv('twitter_training.csv')

In [19]:
vg.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [20]:
column_names = vg.columns

column_names

Index(['2401', 'Borderlands', 'Positive',
       'im getting on borderlands and i will murder you all ,'],
      dtype='object')

In [21]:
print(vg['im getting on borderlands and i will murder you all ,'])

0        I am coming to the borders and I will kill you...
1        im getting on borderlands and i will kill you ...
2        im coming on borderlands and i will murder you...
3        im getting on borderlands 2 and i will murder ...
4        im getting into borderlands and i can murder y...
                               ...                        
74676    Just realized that the Windows partition of my...
74677    Just realized that my Mac window partition is ...
74678    Just realized the windows partition of my Mac ...
74679    Just realized between the windows partition of...
74680    Just like the windows partition of my Mac is l...
Name: im getting on borderlands and i will murder you all ,, Length: 74681, dtype: object


In [22]:
vg.rename(columns={'2401': 'Sentiment_Batch', 'Borderlands': 'Video_Game', 'Positive': 'Sentiment', 'im getting on borderlands and i will murder you all ,': 'Tweet'}, inplace=True)

In [23]:
vg.head()

,Sentiment_Batch,Video_Game,Sentiment,Tweet
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


In [24]:
vg.describe()

,Sentiment_Batch
count,74681.000000
mean,6432.640149
std,3740.423819
min,1.000000
25%,3195.000000
50%,6422.000000
75%,9601.000000
max,13200.000000


# Preprocessing

In [25]:
# Drop Missing Data
vg.dropna(inplace=True)

# Drop Duplicates
vg.drop_duplicates()


,Sentiment_Batch,Video_Game,Sentiment,Tweet
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...
...,...,...,...,...
74676,9200,Nvidia,Positive,Just realized that the Windows partition of my...
74677,9200,Nvidia,Positive,Just realized that my Mac window partition is ...
74678,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...
74679,9200,Nvidia,Positive,Just realized between the windows partition of...


In [34]:
values_to_delete = ['Nvidia', 'johnson&johnson','Facebook', 'HomeDepot', 'Verizon', 'Microsoft', 'Google', 'PlayStation5(PS5)', 'Xbox(Xseries)', 'Amazon']
filtered_vg = vg[~vg['Video_Game'].isin(values_to_delete)]

In [37]:
filtered_vg.iloc[4655:4659]

,Sentiment_Batch,Video_Game,Sentiment,Tweet
7010,9206,Overwatch,Negative,Im not bad at overwatch im terrible 🤣🤣 pic.twi...
7011,9206,Overwatch,Negative,2 Im not bad at overwatch im terrible 🤣🤣 pic.t...
7012,9206,Overwatch,Negative,can
7013,9207,Overwatch,Neutral,I'm WHEEZING


In [40]:
# text cleaning
def clean_text(text):
    if isinstance(text, str):
        text = re.sub(r'http\S+|www\S+|https\S+|pic\S+', '', text, flags=re.MULTILINE)  # remove URLs
        text = re.sub(r'@\w+', '', text)  # remove mentions
        text = re.sub(r'\s*@+\s*', ' ', text) # remove mentions with extra spaces
        text = re.sub(r'#\w+', '', text)  # remove hashtags
        text = re.sub(r'\s+', ' ', text)  # remove extra spaces
        text = re.sub(r'-', '', text) #remove hyphens
        text = re.sub(r'[><]', '', text) #remove greater/less than signs
        text = re.sub(r'[\\/]', '', text) #remove /\ signs
        text = re.sub(r'\.', '', text) # remove "."
        text = re.sub(r'\,', '', text) #remove ","
        text = re.sub(r'unk', '', text) # remove "unk"        
        return text.strip()
    else:
        return text
    

filtered_vg['Clean_Text'] = filtered_vg['Tweet'].apply(clean_text)

/tmp/ipykernel_82/606862506.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_vg['Clean_Text'] = filtered_vg['Tweet'].apply(clean_text)


In [42]:
print(filtered_vg.iloc[3210:3214, 4])

3230                          Love to see :)
3231                    Love to still see it
3232                            it to see it
3233    I’m very impatiently waiting to play
Name: clean_text, dtype: object


# Analysis

# Visualization

# Results/Findings

# Contribution